In [ ]:
import numpy as np
import pandas as pd

import plotly.express as px #for visualization
import matplotlib.pyplot as plt #for visualization


from sklearn.preprocessing import LabelEncoder

#Creating data frame by reading dataset
data_df = pd.read_excel("customer_churn_large_dataset.xlsx")

#Data Cleaning
data_df.dropna(inplace=True)  # Remove rows with missing values

data_df.drop_duplicates(inplace=True)  # Remove duplicate rows


gender_mapping = {'Male': 1, 'Female': 0}
location_mapping = {'Los Angeles': 0, 'Chicago': 1, 'Miami': 2, 'New York': 3, 'Houston': 4}

# Apply gender mapping to the 'Gender' column
data_df['Gender'] = data_df['Gender'].map(gender_mapping)

# Apply location mapping to the 'Location' column
data_df['Location'] = data_df['Location'].map(location_mapping)

data_df.drop(["CustomerID"],axis=1,inplace = True)

print(data_df.head())


In [ ]:
#feature scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
# data_df['Age'] = sc.fit_transform(data_df[['Age']])
# data_df['Location'] = sc.fit_transform(data_df[['Location']])
data_df['Subscription_Length_Months'] = sc.fit_transform(data_df[['Subscription_Length_Months']])
data_df['Monthly_Bill'] = sc.fit_transform(data_df[['Monthly_Bill']])
data_df['Total_Usage_GB'] = sc.fit_transform(data_df[['Total_Usage_GB']])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#Import metric for performance evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import GridSearchCV



#Split data into train and test sets
from sklearn.model_selection import train_test_split
X = data_df.drop('Churn', axis=1)
y = data_df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

#Defining the modelling function
def modeling(alg, alg_name, params={}):
    model = alg(**params) #Instantiating the algorithm class and unpacking parameters if any
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    #Performance evaluation
    def print_scores(alg, y_true, y_pred):
        print(alg_name)
        acc_score = accuracy_score(y_true, y_pred)
        print("accuracy: ",acc_score)
        pre_score = precision_score(y_true, y_pred)
        print("precision: ",pre_score)
        rec_score = recall_score(y_true, y_pred)
        print("recall: ",rec_score)
        f_score = f1_score(y_true, y_pred, average='weighted')
        print("f1_score: ",f_score)

    print_scores(alg, y_test, y_pred)
    return model

# Running logistic regression model
log_model = modeling(LogisticRegression, 'Logistic Regression')

In [ ]:
#Random forest
rf_model = modeling(RandomForestClassifier, "Random Forest Classification")

In [ ]:
#Decision tree
dt_model = modeling(DecisionTreeClassifier, "Decision Tree Classification")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow import keras
from tensorflow.keras import layers

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a Deep Neural Network (DNN)
model = keras.Sequential()
model.add(layers.Input(shape=(X_train_scaled.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Define a grid of hyperparameters to search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

# Create a logistic regression model
model = LogisticRegression()

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)


from sklearn.linear_model import LogisticRegression

# Instantiate a logistic regression model with the best hyperparameters
best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'])

# Train the model on your full training dataset
best_model.fit(X_train, y_train)


from sklearn.metrics import accuracy_score, classification_report

# Make predictions on the test dataset
y_pred = best_model.predict(X_test)

# Calculate accuracy and other relevant metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Model Accuracy:", accuracy)
print("Classification Report:\n", report)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

# Perform 5-fold cross-validation
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("Cross-Validation Accuracy:", scores.mean())

In [ ]:
#Saving best model
import joblib
#Sava the model to disk
filename = 'model.sav'
print(log_model)
joblib.dump(log_model, filename)